# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('weather_data.csv')
city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Humidity,Temperature,Cloudiness,Wind Speed
0,0,Harindanga,22.04,88.32,84.0,87.80,75.0,4.70
1,1,Punta Arenas,-53.16,-70.91,87.0,42.80,75.0,4.70
2,2,Bethel,60.79,-161.76,69.0,39.20,75.0,6.93
3,3,Sambhar,26.91,75.18,58.0,86.00,75.0,6.93
4,4,Concepcion,9.56,-73.66,97.0,73.89,1.0,16.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [14]:
#Create a heatmap for humidity in the cities
m = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(city_df[["Latitude", "Longitude"]], weights=city_df['Humidity'],
             max_intensity=100, point_radius=3, dissipating=False)

m.add_layer(heatmap_layer)

m

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#Create a dataframe of cities with the ideal weather
ideal_df = city_df[(city_df['Temperature'] < 75) &
                   (city_df['Temperature'] > 60) &
                   (city_df['Wind Speed'] < 10) &
                   (city_df['Humidity'] < 40)].reset_index(drop=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#Create a column for hotel name
ideal_df['Hotel Name'] = ""
ideal_df

,Unnamed: 0,City,Latitude,Longitude,Humidity,Temperature,Cloudiness,Wind Speed,Hotel Name
0,63,Half Moon Bay,37.46,-122.43,34.0,66.20,40.0,7.96,
1,259,Yumen,40.29,97.04,26.0,60.39,31.0,9.42,
2,400,Talas,42.52,72.24,33.0,60.70,20.0,8.01,
3,403,Tigzirt,36.57,4.20,38.0,60.83,20.0,4.70,


In [17]:
#Base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up parameters dictionary for api call
params = {
    'rankby': 'distance',
    'keyword': 'hotel',
    'key': g_key
}

#Create loop to retrieve latitude and longitude data and run api call

for index, row in ideal_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    
    #Add location to parameters
    
    params['location'] = f"{lat},{lon}"

# API request   
    hotel_json = requests.get(base_url, params=params).json()
    
#Adding hotel to dataframe

    if hotel_json['status'] == 'OK':
        ideal_df.loc[index, 'Hotel Name'] = hotel_json['results'][0]['name']
    else:
        pass

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = _df[["Latitude", "Longitude"]]

KeyError: "None of [Index(['Lat', 'Long'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map
#Create a map with markers for hotels
hotel_layer = gmaps.marker_layer(
    ideal_df[["Latitude", "Longitude"]],
    info_box_content=hotel_info
)

fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig


# Display Map
fig = gmaps.figure()

fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)

fig